需求：
1. 表格之间先合并
2. 不同的样本（q1,q1）之间如果Hit Name一样，且RT(min)相差在std<0.02，就合并这个峰（其中quality取最大，Area取平均值）

# 加载数据，简单处理

目前无法自动给q1和q2添加samples的特征，后续需要再改

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def process_dataframe(df):
    #  删除不要的信息
    df.columns = df.iloc[7]
    df.drop(df.index[[0,1,2,3,4,5,6,7]],inplace=True)  
    df.drop(['Compound number (#)','Scan number (#)','Baseline Heigth (Ab)','Absolute Heigth (Ab)',\
             'Peak Width 50% (min)','Hit Number','Mol Weight (amu)','CAS Number','Library','Entry Number Library'],axis = 1,inplace=True) 
    # 填充信息
    df['RT (min)'] = df['RT (min)'].ffill()
    df['Area (Ab*s)'] = df['Area (Ab*s)'].ffill()
    #df = df[df['Quality']>70]# 删去低质量的值
    df = df.groupby('RT (min)').head(1) # 根据RT来筛选出前1个Hit Name
    df['sample'] = 'df0'
    return df

df_q1 = pd.read_excel('MSRep-q1.xls',sheet_name='LibRes')
df_q1 = process_dataframe(df_q1)
df_q1['samples'] = 'q1'
df_q2 = pd.read_excel('MSRep-q2.xls',sheet_name='LibRes')
df_q2 = process_dataframe(df_q2)
df_q2['samples'] = 'q2'


df = df_q1.append(df_q2) # 合并数据
df = df.sort_values(by='RT (min)')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


删选出Q值>70

In [24]:
df.drop('sample',inplace=True ,axis = 1) 
df.reset_index(drop=True,inplace= True)
df = df[df['Quality']>70]

得到需要处理的预数据

In [25]:
df.head()

7,RT (min),Area (Ab*s),Hit Name,Quality,samples
1,10.184,29056206,"Silanamine, N,N'-methanetetraylbis[1,1,1-trime...",92,q2
13,10.728,13324103,Methyl 2-(tert-butyldimethylsilyloxy)acetate,72,q2
28,11.970,706701,Methyltris(trimethylsiloxy)silane,91,q1
32,12.319,825051,"3,7-Dioxa-2,8-disilanonane, 2,2,8,8-tetramethyl-",90,q1
33,12.336,1033519,"3,7-Dioxa-2,8-disilanonane, 2,2,8,8-tetramethyl-",90,q2


## 筛选出Hit Name 的重复出来

In [26]:
dul = df.loc[df['Hit Name'].duplicated(keep = False)]

## 找出标准差小于0.02的数据 (峰面积相似没有筛选)

In [27]:
dul['std'] =  dul.groupby(['Hit Name'])['RT (min)'].transform(lambda x: x.std())  # 求出对应的std
dul = dul.loc[dul['std'] < 0.02]
re_dul = dul.index  # 获得需要删除的数据的index
dul.reset_index() # 重新索引
df.drop(re_dul, inplace=True) # 删除原始df中的需要合并的数据“”

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# 合并处理

## 峰面积求出平均值

In [28]:
dul['Area (Ab*s)'] = dul.groupby(['Hit Name'])['Area (Ab*s)'].transform(lambda x : x.mean())

In [29]:
dul.head()

7,RT (min),Area (Ab*s),Hit Name,Quality,samples,std
32,12.319,929285.0,"3,7-Dioxa-2,8-disilanonane, 2,2,8,8-tetramethyl-",90,q1,0.012021
33,12.336,929285.0,"3,7-Dioxa-2,8-disilanonane, 2,2,8,8-tetramethyl-",90,q2,0.012021
44,13.732,2367635.0,Tris(trimethylsilyl)hydroxylamine,98,q1,0.007778
45,13.743,2367635.0,Tris(trimethylsilyl)hydroxylamine,96,q2,0.007778
71,17.897,4933637.0,"Silanamine, 1,1,1-trimethyl-N-(trimethylsilyl)...",91,q1,0.004243


Q值排序

In [30]:
sort_dul = dul.groupby(['Hit Name']).apply(pd.DataFrame.sort_values,'Quality')

In [10]:
sort_dul.head()

7                                                       RT (min)  Area (Ab*s)  \
Hit Name                                                                        
.alpha.-D-Glucopyranoside, 1,3,4,6-tetrakis-O-(... 242    45.946    6926383.5   
                                                   243    45.952    6926383.5   
.alpha.-Linolenic acid, trimethylsilyl ester       195    38.290   17286476.5   
                                                   196    38.296   17286476.5   
2-Amino-2-(hydroxymethyl)propane-1,3-diol, N-ac... 120    26.383    4675137.0   

7                                                                                                Hit Name  \
Hit Name                                                                                                    
.alpha.-D-Glucopyranoside, 1,3,4,6-tetrakis-O-(... 242  .alpha.-D-Glucopyranoside, 1,3,4,6-tetrakis-O-...   
                                                   243  .alpha.-D-Glucopyranoside, 1,3,4,6-tetrakis-O-...   
.alpha.-Linolenic acid, trimethylsilyl ester       195       .alpha.-Linolenic acid, trimethylsilyl ester   
                                                   196       .alpha.-Linolenic acid, trimethylsilyl ester   
2-Amino-2-(hydroxymethyl)propane-1,3-diol, N-ac... 120  2-Amino-2-(hydroxymethyl)propane-1,3-diol, N-a...   

7                                                      Quality samples  \
Hit Name                                                                 
.alpha.-D-Glucopyranoside, 1,3,4,6-tetrakis-O-(... 242      91      q2   
                                                   243      91      q1   
.alpha.-Linolenic acid, trimethylsilyl ester       195      99      q2   
                                                   196      99      q1   
2-Amino-2-(hydroxymethyl)propane-1,3-diol, N-ac... 120      95      q2   

7                                                            std  
Hit Name                                                          
.alpha.-D-Glucopyranoside, 1,3,4,6-tetrakis-O-(... 242  0.004243  
                                                   243  0.004243  
.alpha.-Linolenic acid, trimethylsilyl ester       195  0.004243  
                                                   196  0.004243  
2-Amino-2-(hydroxymethyl)propane-1,3-diol, N-ac... 120  0.000000

In [11]:
sort_dul.drop_duplicates(subset=['Hit Name'],keep = 'first',inplace= True)
sort_dul.drop(['std'],axis =1 ,inplace=True)

In [12]:
sort_dul[['RT (min)','Hit Name']]

,7,RT (min),Hit Name
Hit Name,,,
".alpha.-D-Glucopyranoside, 1,3,4,6-tetrakis-O-(trimethylsilyl)-.beta.-D-fructofuranosyl 2,3,4,6-tetrakis-O-(trimethylsilyl)-",242,45.946,".alpha.-D-Glucopyranoside, 1,3,4,6-tetrakis-O-..."
".alpha.-Linolenic acid, trimethylsilyl ester",195,38.290,".alpha.-Linolenic acid, trimethylsilyl ester"
"2-Amino-2-(hydroxymethyl)propane-1,3-diol, N-acetyl-, tris(trimethylsilyl) ether",120,26.383,"2-Amino-2-(hydroxymethyl)propane-1,3-diol, N-a..."
"3,7-Dioxa-2,8-disilanonane, 2,2,8,8-tetramethyl-",32,12.319,"3,7-Dioxa-2,8-disilanonane, 2,2,8,8-tetramethyl-"
"Cyclohexasiloxane, dodecamethyl-",85,19.305,"Cyclohexasiloxane, dodecamethyl-"
Dibutyl phthalate,158,33.593,Dibutyl phthalate
"Glycine, N,N-bis(trimethylsilyl)-, trimethylsilyl ester",81,18.956,"Glycine, N,N-bis(trimethylsilyl)-, trimethylsi..."
"Hexadecanoic acid, 2,3-bis[(trimethylsilyl)oxy]propyl ester",235,44.430,"Hexadecanoic acid, 2,3-bis[(trimethylsilyl)oxy..."
"Hexanedioic acid, bis(2-ethylhexyl) ester",210,41.237,"Hexanedioic acid, bis(2-ethylhexyl) ester"


In [13]:
df.head(10)

7,RT (min),Area (Ab*s),Hit Name,Quality,samples
1,10.184,29056206,"Silanamine, N,N'-methanetetraylbis[1,1,1-trime...",92,q2
13,10.728,13324103,Methyl 2-(tert-butyldimethylsilyloxy)acetate,72,q2
28,11.970,706701,Methyltris(trimethylsiloxy)silane,91,q1
34,12.433,934829,"Tetrasiloxane, decamethyl-",91,q1
43,13.698,1239727,"1-(3-Methylbutyl)-2,3,4,6-tetramethylbenzene",78,q1
66,17.211,1060890,Benzoic acid trimethylsilyl ester,96,q1
77,18.269,19777658,"Glycerol, tris(trimethylsilyl) ether",90,q1
95,22.120,16236925,"Silanamine, 1,1,1-trimethyl-N-propyl-N-(trimet...",78,q1
105,23.562,1088002,Pentadecane,92,q1
108,23.751,2473209,"Butanedioic acid, [(trimethylsilyl)oxy]-, bis(...",76,q2


添加被合并的峰

In [14]:
df1 = df.append(sort_dul)

In [18]:
df1.sort_values(by='RT (min)',inplace=True)
df1 = df1.reset_index(drop=True);df1

7,RT (min),Area (Ab*s),Hit Name,Quality,samples
0,10.184,29056206.0,"Silanamine, N,N'-methanetetraylbis[1,1,1-trime...",92,q2
1,10.728,13324103.0,Methyl 2-(tert-butyldimethylsilyloxy)acetate,72,q2
2,11.970,706701.0,Methyltris(trimethylsiloxy)silane,91,q1
3,12.319,929285.0,"3,7-Dioxa-2,8-disilanonane, 2,2,8,8-tetramethyl-",90,q1
4,12.433,934829.0,"Tetrasiloxane, decamethyl-",91,q1
5,13.698,1239727.0,"1-(3-Methylbutyl)-2,3,4,6-tetramethylbenzene",78,q1
6,13.743,2367635.0,Tris(trimethylsilyl)hydroxylamine,96,q2
7,17.211,1060890.0,Benzoic acid trimethylsilyl ester,96,q1
8,17.897,4933637.0,"Silanamine, 1,1,1-trimethyl-N-(trimethylsilyl)...",91,q1
9,18.224,37137303.0,"Silanol, trimethyl-, phosphate (3:1)",99,q1


In [16]:
writer = pd.ExcelWriter('test.xlsx')

In [17]:
df1.to_excel(writer)
writer.save()